**Predicting the Amount of People Using Bike-Share on a Given Day**  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Bike sharing and rental systems have been introduced across many urban areas over the past few decades, and have had many benefits to urban transportation, such as by providing convenient, eco-friendly, and cost-effective mobility solutions. With the growing emphasis on sustainable transportation, bike sharing has gained significant popularity as a means to reduce traffic, lower carbon emissions, and promote healthier lifestyles.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Through our data analysis project, we would like to figure out the amount of people using bike rental systems on a given day, with respect to temperature. Thus, our question is the following: *What is the relationship between daily temperature and the number of people using bike rental systems, and can we predict the number of users based on this temperature data?* We will be using the Seoul Bike Sharing Demand dataset, which contains the count of public bicycles rented per hour within the Seoul Bike Sharing System, in addition to the corresponding weather data and holiday information.


In [ ]:
library(tidyverse)
library(tidymodels)
install.packages("janitor") # for clean_names
library(janitor)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.2     ✔ purrr   1.0.1
✔ tibble  3.2.1     ✔ dplyr   1.1.1
✔ tidyr   1.3.0     ✔ stringr 1.5.0
✔ readr   2.1.3     ✔ forcats 0.5.2
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.2     ✔ rsample      1.1.1
✔ dials        1.1.0     ✔ tune         1.0.1
✔ infer        1.0.4     ✔ workflows    1.1.2
✔ modeldata    1.0.1     ✔ workflowsets 1.0.0
✔ parsnip      1.0.3     ✔ yardstick    1.1.0
✔ recipes      1.0.4     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks read

In [ ]:
# read data from web
bike_url <- "https://raw.githubusercontent.com/caiwwei/dsci-project-4/main/SeoulBikeData.csv"
bike_data <- read_csv(bike_url) |>
    clean_names() # tidy column names in dataset to contain no spaces

# convert data observations from hourly to daily by computing total daily users and average daily temperature
bike_condensed <- bike_data |>
    group_by(date) |> 
    filter(functioning_day == "Yes") |> # because we only care about days where the bike share service is operating
    summarize(avg_temp = mean(temperature_celsius), total_users = sum(rented_bike_count)) 

# split the data into training and testing; we choose to use a 75% training and 25% testing split 
bike_split <- initial_split(bike_condensed, prop = 0.75, strata = total_users)
bike_training <- training(bike_split)
bike_testing <- (bike_split)

# summarize training data; we calculate the average temperature and the average number of daily users for our training dataset
bike_summary <- bike_training |>
    summarize(avg_daily_temp = mean(avg_temp), avg_daily_users = mean(total_users))
bike_summary

# count of all NA values in the training datset for each column
bike_NA_summary <- bike_training |>
    summarize(avg_temp_NA_count = sum(is.na(avg_temp)), # sums rows that have NA values in given column
              avg_users_NA_count = sum(is.na(total_users)), date_NA_count = sum(is.na(date)))
bike_NA_summary

# visualize data with a scatter plot to depict relationship between average temperature and total users
bike_plot <- ggplot(bike_condensed, aes(x = avg_temp, y = total_users)) +
    geom_point() +
    xlab("Daily Average Temperature (Celsius)") +
    ylab("Daily Total Users") +
    ggtitle("Average Daily Temperature vs. Total Daily Bike Share Users")

bike_plot

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Since this is a regression problem about the relationship between the amount of bike share users and the temperature, we would like to examine only these two variables. We chose temperature in particular since we believe that it is the most informational variable regarding the amount of bike rental system users, as it can cause significant variances depending on day. In addition, since this dataset recorded data at several points throughout the day, and we are only interested in finding out the daily counts, we grouped the temperature by day and took the average. Furthermore, we can then visualize our results with the use of a scatter plot, as it will show the correlations between our variables well. 

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;We expect that the amount of bike share users will be lower towards the higher and lower ends of the temperature spectrum, and will be the highest near the middle at a milder, warmer temperature. Thus, through the process of regression, we will be able to predict the amount of bike share users for a given day with a particular average temperature. Understanding this information can have many positive impacts on the progression of bike sharing systems, such as with allocating resources. For instance, cities or bike-sharing operators will be able to deploy additional bikes or staff during peak demand on mild, warmer days and reduce resources during colder periods or extreme heat, which will help optimize operations and reduce costs.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Moreover, this data analysis can also pose some future questions, such as how to improve the bike-sharing industry and system through our results, as well as with more specific ways to determine bike rental patterns. For example, our analysis could be the basis for the following questions: How do bike rental patterns change with the seasons? Are there significant differences in temperature-related usage between summer and winter? How does bike rental usage respond to extreme temperature conditions, such as heat waves or cold snaps? Are there temperature thresholds that significantly affect usage?